<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/data-augmentation/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import cv2
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations import HorizontalFlip, RandomBrightnessContrast, ShiftScaleRotate
from albumentations import Normalize
from tqdm import tqdm
import numpy as np

d:\PythonProject\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
def rotate(img):
  Cx, Cy = img.shape[:2]
  rand_angle = random.randint(-180,180)
  M = cv.getRotationMatrix2D((Cy//2, Cx//2),rand_angle ,1)
  img = cv.warpAffine(img, M, (Cx, Cy))
  return img

def flip(img):
  num = [0, 1, -1]
  img = cv.flip(img, random.choice(num))
  return img

def noise(img):
  row, col = img.shape[:2]
  number_of_pixels = random.randint(300, 1000)
  for i in range(number_of_pixels):
    y_coord=random.randint(0, row - 1)
    x_coord=random.randint(0, col - 1)
    img[y_coord][x_coord] = 255

  number_of_pixels = random.randint(300 , 1000)
  for i in range(number_of_pixels):
      y_coord=random.randint(0, row - 1)
      x_coord=random.randint(0, col - 1)
      img[y_coord][x_coord] = 0
  return img

In [8]:
transform = A.Compose([
    A.Resize(224, 224),
    A.RandomBrightnessContrast(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.Rotate(limit=15, p=0.5),
])

In [9]:
def augment_and_save(input_dir, output_dir, num_augments=3):
    """
    Creates augmented images from a dataset and saves them to disk.

    Args:
        input_dir (str): Path to the original dataset directory (with class subfolders).
        output_dir (str): Path to save the augmented dataset.
        num_augments (int): Number of augmented images to create per original image.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through each class folder
    for class_name in os.listdir(input_dir):
        if(class_name == 'disgust'):
            continue
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue  # Skip non-folder files

        # Create a corresponding folder in the output directory
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        # Loop through each image in the class folder
        for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            # if(len(os.listdir(f'{output_dir}/{class_name}')) == 3000):
            #     break
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)  # Load the image
            if image is None:
                continue  # Skip unreadable images
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Save original image
            original_output_path = os.path.join(output_class_path, img_name)
            cv2.imwrite(original_output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            # Generate and save augmented images
            for i in range(num_augments):
                # if(len(os.listdir(f'{output_dir}/{class_name}')) == 3000):
                #     break
                augmented = transform(image=image)['image']  # Apply augmentation
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"
                aug_img_path = os.path.join(output_class_path, aug_img_name)
                cv2.imwrite(aug_img_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

In [10]:
input_dir = "C:\\Users\\vinar\\Downloads\\FER2013-down-sampling"
output_dir = "C:\\Users\\vinar\\Downloads\\FER2013-augmented"
augment_and_save(input_dir, output_dir, num_augments=3)

Processing surprise: 100%|██████████| 3000/3000 [00:19<00:00, 150.35it/s]
